In [1]:
import numpy as np
import cv2
from collections import deque
import mediapipe as mp
# from tensorflow.keras.saving import load_model
import tensorflow.lite as lite

In [2]:
SEQ_LEN = 10

CLASSES_LIST =  ['Fall Down',
 'Sitting down',
 'Walking',
 'Lying down',
 'Standing up',
 'Standing',
 'Sitting',
 'Chest Pain',
 'Coughing']

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.7, min_tracking_confidence=0.7, model_complexity=1)
mp_draw = mp.solutions.drawing_utils


In [4]:
def tflite_predict(X, interpreter):
    interpreter.allocate_tensors()
    interpreter.set_tensor(interpreter.get_input_details()[0]['index'], np.expand_dims(X, 0))
    interpreter.invoke()
    predictions = interpreter.get_tensor(interpreter.get_output_details()[0]['index'])
    return predictions

In [5]:
def pred_video(interpreter, vid=0 ,SEQUENCE_LENGTH=SEQ_LEN):
    cap = cv2.VideoCapture(vid)
    frames_queue = deque(maxlen = SEQUENCE_LENGTH)
    predicted_class_name = ''
    while cap.isOpened():
        ret, org_frame = cap.read()
        if not ret:
            break
        org_frame = cv2.flip(org_frame, 1)

        frame = cv2.resize(org_frame, (256, 256))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        results = pose.process(frame)

        pose_lm = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten()  if results.pose_landmarks else np.zeros(132)
        
        mp_draw.draw_landmarks(org_frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS) 

        frames_queue.append(pose_lm.astype('float32'))

        if len(frames_queue) == SEQUENCE_LENGTH:
            
            predicted_labels_probabilities = tflite_predict(frames_queue, interpreter)
            
            predicted_label = np.argmax(predicted_labels_probabilities)

            predicted_class_name = CLASSES_LIST[predicted_label]


        cv2.putText(org_frame, predicted_class_name, (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 5) if results.pose_landmarks else cv2.putText(org_frame, 'No Action', (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 5)

        cv2.imshow('frame', org_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            break
    cap.release()
    cv2.destroyAllWindows()

    

In [6]:
TF_LITE_MODEL_NAME = 'final_model.tflite'
interpreter = lite.Interpreter(TF_LITE_MODEL_NAME)

vid_path = 0
pred_video(interpreter, vid=vid_path)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
